In [1]:
!pip install langchainhub

  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/e9/4c/25e098d3f43a57e3d0936ca09c4ad607fd40ac4b784072ccdbd4dfa55967/langchainhub-0.1.13-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/1e/50/fb7eed30eace72d2eb6cf71059f896e7b4a0c031b654b99f37146a68d60b/types_requests-2.31.0.8-py3-none-any.whl.metadata


In [2]:
from langchain import hub

In [4]:
prompt = hub.pull("hwchase17/react-chat")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descr

In [5]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [10]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=10e99242b440abcc6504675aecfdb6ac9f25222c7de4ce0f994d9f07b15fbd90
  Stored in directory: /home/vietthangif/.cache/pip/wheels/68/8e/73/744b7d9d7ac618849d93081a20e1c0deccd2aef90901c9f5a9
Successfully built google-search-results


In [11]:
# set SERP API key
import os
os.environ["SERPAPI_API_KEY"] = "2247ec2442f51de10c72d824d516af3d8c012eb5eeb86420eab4058e1aa7dab8"

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [16]:
from langchain.tools.render import render_text_description
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'chat_history'], partial_variables={'tools': 'Current Search: useful for when you need to answer questions about current events or the current state of the world', 'tool_names': 'Current Search'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Add

In [32]:
llm=OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
llm_with_stop = llm.bind(stop=["\nObservation"])

In [28]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

In [29]:
from langchain.agents import AgentExecutor

In [30]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [31]:
agent_executor.invoke({"input": "hi, i am bob"})['output']



> Entering new AgentExecutor chain...


InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [26]:
llm.model_name

'text-davinci-003'